## 1.3 LCEL(LangChain Expression Language) - Advanced

<div style="text-align: right"> Initial issue : 2025.04.26 </div>
<div style="text-align: right"> last update : 2025.04.26 </div>

`Runnable이란`  
- Langchain에서 워크플로우나 체인 구성을 위해 사용하는 프로토콜(인터페이스)
- 모델 관련 컴포넌트(프롬프트, LLM, 출력파서, 리트리버 등)를 표준화된 방식으로 조합하고 실행하도록 하는 역할

`Runnable` 객체의 표준 인터페이스 종류
- [`stream`](#stream): 응답의 청크를 스트리밍.
- [`invoke`](#invoke): 입력에 대해 체인을 호출.
- [`batch`](#batch): 입력 목록에 대해 체인을 호출.   
            
비동기 메서드도 존재
- [`astream`](#async-stream): 비동기적으로 응답의 청크를 스트리밍.
- [`ainvoke`](#async-invoke): 비동기적으로 입력에 대해 체인을 호출.
- [`abatch`](#async-batch): 비동기적으로 입력 목록에 대해 체인을 호출.
- [`astream_log`](#async-stream-intermediate-steps): 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from utils import langsmith
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

### Chain 생성하기

In [3]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
prompt = PromptTemplate.from_template(
    "{topic}에 대해서 3문장으로 설명하세요."
)
chain = prompt | model | StrOutputParser()

### stream: 실시간 출력   
- 주어진 토픽에 대한 데이터 스트림을 생성
- 이 스트림을 반복하여 각 데이터의 내용(`content`)을 즉시 출력합니다. 
- `end=""` 인자는 출력 후 줄바꿈을 하지 않도록 설정
- `flush=True` 인자는 출력 버퍼를 즉시 비우도록 합니다. 

In [4]:
for token in chain.stream({"topic": "멀티모달"}):
    print(token, end="", flush=True)

멀티모달은 다양한 형태의 데이터를 동시에 처리하고 분석하는 기술을 의미합니다. 예를 들어, 텍스트, 이미지, 오디오 등 서로 다른 유형의 정보를 결합하여 더 풍부한 이해를 도출할 수 있습니다. 이러한 접근 방식은 인공지능과 머신러닝 분야에서 특히 중요하며, 자연어 처리와 컴퓨터 비전의 융합을 통해 다양한 응용 프로그램에 활용됩니다.

### invoke: 체인 호출   
- 주제를 인자로 받아 해당 주제에 대한 처리 수행

In [5]:
chain.invoke({"topic": "ChatGPT"})

'ChatGPT는 OpenAI가 개발한 대화형 인공지능 모델로, 자연어 처리 기술을 기반으로 합니다. 사용자의 질문이나 요청에 대해 이해하고 적절한 답변을 생성하여 대화할 수 있는 능력을 가지고 있습니다. 다양한 주제에 대해 정보를 제공하고, 창의적인 글쓰기, 문제 해결 등 여러 용도로 활용될 수 있습니다.'

### batch: 배치 처리   
- 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아 일괄 처리를 수행

In [6]:
chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])

['ChatGPT는 OpenAI가 개발한 대화형 인공지능 모델로, 자연어 처리 기술을 기반으로 합니다. 사용자의 질문이나 요청에 대해 인간처럼 자연스럽고 유창한 대화를 생성할 수 있습니다. 다양한 주제에 대한 정보 제공, 문제 해결, 창의적인 글쓰기 등 여러 용도로 활용될 수 있습니다.',
 '인스타그램은 사용자들이 사진과 동영상을 공유할 수 있는 소셜 미디어 플랫폼입니다. 다양한 필터와 편집 도구를 제공하여 사용자가 자신의 콘텐츠를 창의적으로 표현할 수 있도록 돕습니다. 또한, 친구와의 소통, 해시태그를 통한 관심사 탐색, 그리고 브랜드와의 연결을 통해 커뮤니티를 형성하는 데 중요한 역할을 합니다.']

- `max_concurrency` 매개변수를 사용하여 동시 요청 수를 설정할 수 있음
- `config` 딕셔너리는 `max_concurrency` 키를 통해 동시에 처리할 수 있는 최대 작업 수를 설정

In [7]:
chain.batch(
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "멀티모달"},
        {"topic": "프로그래밍"},
        {"topic": "머신러닝"},
    ],
    config={"max_concurrency": 3},
)

['ChatGPT는 OpenAI가 개발한 대화형 인공지능 모델로, 자연어 처리 기술을 기반으로 합니다. 사용자의 질문이나 요청에 대해 인간처럼 자연스럽고 유창한 대화를 생성할 수 있습니다. 다양한 주제에 대한 정보 제공, 문제 해결, 창의적인 글쓰기 등 여러 용도로 활용될 수 있습니다.',
 '인스타그램은 사용자들이 사진과 동영상을 공유할 수 있는 소셜 미디어 플랫폼입니다. 다양한 필터와 편집 도구를 제공하여 사용자가 자신의 콘텐츠를 창의적으로 표현할 수 있도록 돕습니다. 또한, 친구와의 소통, 해시태그를 통한 관심사 탐색, 그리고 브랜드와의 연결을 통해 커뮤니티를 형성하는 데 중요한 역할을 합니다.',
 '멀티모달은 다양한 형태의 데이터를 동시에 처리하고 분석하는 기술을 의미합니다. 예를 들어, 텍스트, 이미지, 음성 등의 여러 모드를 결합하여 보다 풍부한 정보를 추출하고 이해할 수 있습니다. 이러한 접근 방식은 인공지능 분야에서 특히 중요하며, 자연어 처리와 컴퓨터 비전의 융합을 통해 더 나은 성능을 발휘합니다.',
 '프로그래밍은 컴퓨터가 특정 작업을 수행하도록 지시하는 과정으로, 주로 프로그래밍 언어를 사용하여 코드를 작성합니다. 이 과정에서는 알고리즘을 설계하고, 문제를 해결하기 위한 논리를 구성하며, 소프트웨어를 개발하는 데 필요한 다양한 기술을 활용합니다. 결과적으로 프로그래밍은 웹사이트, 애플리케이션, 게임 등 다양한 디지털 제품을 만드는 데 필수적인 역할을 합니다.',
 '머신러닝은 데이터에서 패턴을 학습하여 예측이나 결정을 자동으로 수행하는 인공지능의 한 분야입니다. 알고리즘을 사용하여 주어진 데이터로부터 모델을 생성하고, 이 모델을 통해 새로운 데이터에 대한 분석을 수행합니다. 머신러닝은 이미지 인식, 자연어 처리, 추천 시스템 등 다양한 응용 분야에서 활용되고 있습니다.']

### async stream: 비동기 스트리밍   
- 주어진 토픽에 대한 메시지를 비동기적으로 처리
- 비동기 for 루프(`async for`)를 사용하여 스트림에서 메시지를 순차적으로 받아오고, 메시지의 내용을 즉시 출력
- `end=""`는 출력 후 줄바꿈을 하지 않도록 설정하며, `flush=True`는 출력 버퍼를 강제로 비워 즉시 출력되도록 함

In [8]:
async for token in chain.astream({"topic": "YouTube"}):
    print(token, end="", flush=True)

YouTube는 사용자들이 동영상을 업로드, 공유 및 시청할 수 있는 세계 최대의 비디오 플랫폼입니다. 다양한 콘텐츠가 제공되며, 개인 사용자부터 기업, 크리에이터까지 다양한 사람들이 활동하고 있습니다. 또한, YouTube는 광고 수익, 구독 서비스 및 라이브 스트리밍 기능 등을 통해 수익을 창출하는 플랫폼이기도 합니다.

### async invoke: 비동기 호출
- 비동기적으로 주어진 인자를 사용하여 작업을 수행

In [9]:
# 비동기 체인 객체의 'ainvoke' 메서드를 호출하여 'NVDA' 토픽처리
my_process = chain.ainvoke({"topic": "NVDA"})

In [10]:
# 비동기로 처리되는 프로세스가 완료될 때까지 기다림
await my_process

'NVIDIA Corporation(NVDA)은 그래픽 처리 장치(GPU)와 인공지능(AI) 기술을 전문으로 하는 미국의 기술 회사입니다. 이 회사는 게임, 데이터 센터, 자율주행차 및 AI 연구 등 다양한 분야에서 사용되는 고성능 컴퓨팅 솔루션을 제공합니다. 최근 몇 년간 AI와 머신러닝의 발전으로 인해 NVIDIA의 제품 수요가 급증하며, 시장에서 중요한 위치를 차지하고 있습니다.'

### async batch: 비동기 배치
- 비동기적으로 일련의 작업을 일괄 처리
- `await` 키워드는 해당 비동기 작업이 완료될 때까지 기다리는데 사용

In [11]:
# 주어진 토픽에 대해 비동기적으로 일괄 처리를 수행
my_abatch_process = chain.abatch(
    [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
)

In [12]:
await my_abatch_process

['YouTube는 사용자들이 동영상을 업로드, 공유 및 시청할 수 있는 세계 최대의 비디오 플랫폼입니다. 다양한 콘텐츠가 제공되며, 개인 사용자부터 기업, 크리에이터까지 다양한 사람들이 활동하고 있습니다. 또한, YouTube는 광고 수익, 구독 서비스 및 라이브 스트리밍 기능 등을 통해 수익을 창출하는 플랫폼이기도 합니다.',
 '인스타그램은 사용자들이 사진과 동영상을 공유하고 소통할 수 있는 소셜 미디어 플랫폼입니다. 다양한 필터와 편집 도구를 제공하여 사용자가 자신의 콘텐츠를 창의적으로 표현할 수 있도록 돕습니다. 또한, 해시태그와 스토리 기능을 통해 사용자 간의 연결과 발견을 촉진합니다.',
 'Facebook은 2004년에 마크 저커버그와 그의 동료들이 설립한 소셜 미디어 플랫폼으로, 사용자들이 친구 및 가족과 소통하고 정보를 공유할 수 있는 공간입니다. 사용자는 프로필을 만들고, 사진과 동영상을 게시하며, 다양한 그룹과 페이지에 참여할 수 있습니다. Facebook은 전 세계적으로 수억 명의 사용자를 보유하고 있으며, 광고와 마케팅을 위한 중요한 플랫폼으로도 자리잡고 있습니다.']